In [69]:
import pandas as pd

In [70]:
df = pd.read_csv('assets/TDS_Week_5_Dataset_1 - Sheet1.csv')
df['Failure Type'].unique()
#df['Failure Type'].count()-df[df['Failure Type'] == 'Power Failure']['Failure Type'].count()

array(['No Failure', 'Power Failure', 'Tool Wear Failure',
       'Overstrain Failure', 'Random Failures',
       'Heat Dissipation Failure'], dtype=object)

In [71]:
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,No Failure


In [72]:
df = pd.get_dummies(df, columns=['Failure Type'], drop_first=True)

In [73]:
df[df['Failure Type_Random Failures'] == 1]['UDI'].count()

18

In [74]:
q_hi = df['Rotational speed [rpm]'].quantile(0.75)
q_low = df['Rotational speed [rpm]'].quantile(0.25)
IQR = q_hi - q_low
lower_range = q_low - (1.5 * IQR)
upper_range = q_hi + (1.5 * IQR)
df= df[(df['Rotational speed [rpm]']>lower_range) & (df['Rotational speed [rpm]']<upper_range)]
#df= df[(df['CNT_CHILDREN'] < 8)]
#df= df[(df['CNT_FAM_MEMBERS'] < 8)]

df.shape

(9582, 13)

In [115]:
import numpy as np
from sklearn.cluster import KMeans

In [153]:
stock = pd.read_csv('assets/TDS_Week_5_Dataset_2 - Sheet1.csv')

In [146]:
stock.head()

,record number,Price,Market Cap,Free Float Market Cap %,6m ADV,RoE %,RoCE %,EBIT Margin %,EPS,PAT %,Stock Return %
0,0,0.001319,0.001970,0.826221,0.021519,0.000000,0.476845,0.038117,0.000701,0.0,0.268651
1,1,0.000769,0.000579,0.441647,0.003081,0.000000,0.232072,0.000000,0.025832,0.0,0.190937
2,2,0.000132,0.000316,0.655003,0.000778,0.065568,0.000000,0.000000,0.001083,0.0,0.122265
3,3,0.000824,0.007208,0.000533,0.000252,0.408417,0.416031,0.000000,0.000918,0.0,0.100503
4,4,0.000596,0.000358,0.805526,0.003378,0.116626,0.474160,0.067420,0.029651,0.0,0.050376


In [154]:
stock['Free Float Market Cap %'] = stock['Free Float Market Cap %']*100
stock['RoE %'] = stock['RoE %']*100
stock['RoCE %'] = stock['RoCE %']*100
stock['EBIT Margin %'] = stock['EBIT Margin %']*100
stock['PAT %'] = stock['PAT %']*100

In [155]:
stock.describe()

,record number,Price,Market Cap,Free Float Market Cap %,6m ADV,RoE %,RoCE %,EBIT Margin %,EPS,PAT %,Stock Return %
count,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000
mean,612.000000,0.010163,0.015214,38.659175,0.027942,52.906076,56.741580,53.847632,0.022321,51.524376,0.145038
std,353.771348,0.038309,0.060153,16.673399,0.090669,12.632005,7.923857,8.536269,0.051913,8.710919,0.075296
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,306.000000,0.001167,0.000601,24.450608,0.000550,50.696507,52.707946,51.687277,0.003966,50.327121,0.102570
50%,612.000000,0.003291,0.001707,36.398549,0.002338,54.490617,55.832344,53.847632,0.009215,51.967712,0.132243
75%,918.000000,0.008680,0.006294,47.973117,0.010388,58.547111,59.838861,56.672358,0.020459,54.324236,0.171613
max,1224.000000,1.000000,1.000000,100.000000,1.000000,100.000000,100.000000,100.000000,1.000000,100.000000,1.000000


In [156]:
feature = stock.columns[1:]
stock[feature].head()

,Price,Market Cap,Free Float Market Cap %,6m ADV,RoE %,RoCE %,EBIT Margin %,EPS,PAT %,Stock Return %
0,0.001319,0.001970,82.622146,0.021519,0.000000,47.684462,3.811693,0.000701,0.0,0.268651
1,0.000769,0.000579,44.164711,0.003081,0.000000,23.207214,0.000000,0.025832,0.0,0.190937
2,0.000132,0.000316,65.500320,0.000778,6.556798,0.000000,0.000000,0.001083,0.0,0.122265
3,0.000824,0.007208,0.053339,0.000252,40.841705,41.603133,0.000000,0.000918,0.0,0.100503
4,0.000596,0.000358,80.552592,0.003378,11.662570,47.416033,6.742022,0.029651,0.0,0.050376


In [157]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
stockDataFeatures_scaled = scaler.fit_transform(stock[feature])
stockDataFeatures_scaled = pd.DataFrame(stockDataFeatures_scaled, columns=feature)
stockDataFeatures_scaled.describe()

,Price,Market Cap,Free Float Market Cap %,6m ADV,RoE %,RoCE %,EBIT Margin %,EPS,PAT %,Stock Return %
count,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000
mean,0.010163,0.015214,0.386592,0.027942,0.529061,0.567416,0.538476,0.022321,0.515244,0.145038
std,0.038309,0.060153,0.166734,0.090669,0.126320,0.079239,0.085363,0.051913,0.087109,0.075296
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.001167,0.000601,0.244506,0.000550,0.506965,0.527079,0.516873,0.003966,0.503271,0.102570
50%,0.003291,0.001707,0.363985,0.002338,0.544906,0.558323,0.538476,0.009215,0.519677,0.132243
75%,0.008680,0.006294,0.479731,0.010388,0.585471,0.598389,0.566724,0.020459,0.543242,0.171613
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [158]:
kmeans = KMeans(n_clusters=7, random_state=1729)

In [159]:
cluster_ = kmeans.fit_predict(stockDataFeatures_scaled)

In [160]:
stock['cluster'] = cluster_
stock['cluster'].value_counts()

0    435
3    382
2    153
1    152
5     62
4     21
6     20
Name: cluster, dtype: int64

In [161]:
print((stock[stock['record number'] == 213].cluster, stock[stock['record number'] == 593].cluster))
print((stock[stock['record number'] == 294].cluster, stock[stock['record number'] == 832].cluster))
print((stock[stock['record number'] == 148].cluster, stock[stock['record number'] == 660].cluster))
print((stock[stock['record number'] == 159].cluster, stock[stock['record number'] == 562].cluster))


(213    3
Name: cluster, dtype: int32, 593    1
Name: cluster, dtype: int32)
(294    3
Name: cluster, dtype: int32, 832    0
Name: cluster, dtype: int32)
(148    3
Name: cluster, dtype: int32, 660    3
Name: cluster, dtype: int32)
(159    3
Name: cluster, dtype: int32, 562    1
Name: cluster, dtype: int32)


In [162]:
clusterDesc = pd.DataFrame(stock.iloc[:,1:].groupby('cluster').mean().round(3))
clusterDesc.insert(0,'size',stock['cluster'].value_counts())

In [163]:
clusterDesc

,size,Price,Market Cap,Free Float Market Cap %,6m ADV,RoE %,RoCE %,EBIT Margin %,EPS,PAT %,Stock Return %
cluster,,,,,,,,,,,
0,435,0.011,0.009,23.182,0.010,55.128,57.174,54.803,0.017,53.027,0.148
1,152,0.013,0.010,67.023,0.031,52.780,55.322,54.045,0.036,51.631,0.131
2,153,0.017,0.023,41.530,0.043,65.377,66.890,60.301,0.006,57.766,0.145
3,382,0.006,0.005,42.546,0.015,52.700,54.903,53.268,0.026,51.391,0.151
4,21,0.021,0.355,50.117,0.565,61.630,64.553,57.119,0.013,54.968,0.174
5,62,0.002,0.003,39.887,0.012,13.052,46.075,45.999,0.051,38.391,0.121
6,20,0.002,0.001,47.677,0.002,28.450,40.477,14.170,0.014,9.945,0.114
